In [ ]:
from main_library import *
eic = unzip_zip(ZIPNAME, f"{PATH_SOURCE}{FRAMEWORK}/", 'proposals_eicFundPortfolio.json', 'utf8')

In [ ]:
eic=pd.DataFrame(eic)

In [ ]:
eic.sort_values('proposalNbr')
eic.head(5
         )

In [ ]:
"https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe_en"
"https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe/cluster-1-health_en"

In [ ]:
"https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe/cluster-1-health_en"

In [ ]:
from config_path import PATH_SOURCE
import time, pandas as pd, os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox .service import Service as FirefoxService
from webdriver_manager.firefox  import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

def get_data_from_html(soup):
    response=[]
    for res in soup.find_all('eui-card-header-subtitle'):
#         print(res.find_all('span')[0].text)
#         print(res.find_all('strong')[0].text)
#         print(res.find_all('strong')[1].text)
        response.append({'topic_code':res.find_all('span')[0].text, 
                        'type':res.find_all('span')[2].text, 
                        'open_date':res.find_all('strong')[0].text, 
                        'deadline':res.find_all('strong')[1].text})
    return response

def click_next(b):
    x = b[2] # 3e element qui convient (après test)
    c = x.find_element(By.CLASS_NAME, 'eui-button')
    c.click()


# def get_topic_info_europa(prog):
prog='cluster-1-health'
url = f"https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe/{prog}_en"

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.maximize_window()
driver.get(url)
time.sleep(5)  

wait = WebDriverWait(driver, 1)
wait.until(EC.presence_of_element_located((By.ID,'cookie-consent-banner')))
cookie = driver.find_element(By.CLASS_NAME, 'wt-ecl-button')
cookie.click()

# # click on preview
b = driver.find_elements(By.CLASS_NAME, 'wt-ecl-link__label')
y=b[0]
y.click()



html_page = driver.page_source
soup = BeautifulSoup(html_page, 'html.parser')
soup.find(class_="wt-modal--content").find('div', attrs={"data-page-number":"2"})



    # counter = soup.find('div', class_='eui-paginator__page-range').get_text().strip().replace(',','').replace('–',' ').split(' ')
    #     print(counter)
    #     new_data = get_data_from_html(soup)
    #     print(len(new_data))
    #     data += new_data
    #     if counter[-3] == counter[-1]:
    #         break
    #     print('---')
    #     b = driver.find_elements(By.CLASS_NAME, 'eui-paginator__page-navigation-item')
    #     click_next(b)
    #     time.sleep(5)
    # #     except:
    # #         wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#cdk-overlay-0 button.btn"))).click()
    # #         driver.execute_script("arguments[0].click()", driver.find_element_by_css_selector("#StudentSatisfactionPop button.btn"))

    # pd.to_pickle(data, open(f"{PATH_SOURCE}{FRAMEWORK}/topic_info_harvest.pkl", 'wb'))

In [ ]:

from config_path import PATH_SOURCE, PATH_WP
import time, pandas as pd, requests, fitz, re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox .service import Service as FirefoxService
from webdriver_manager.firefox  import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


def wp_load(url, year, files_to_load):
  
    driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
    driver.maximize_window()
    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 1)
    wait.until(EC.presence_of_element_located((By.ID,'cookie-consent-banner')))
    cookie = driver.find_element(By.CLASS_NAME, 'wt-ecl-button')
    cookie.click()

    elements = driver.find_elements(By.XPATH, '//a[@data-wt-preview="pdf" and @href and @data-untranslated-label]')

    # Loop through elements and check data-untranslated-label content
    for el in elements:

        label = el.get_attribute("data-untranslated-label")
        href = el.get_attribute("href")
        print(f"{label} -> {href}")
        for key, value in files_to_load.items():
            if key in re.sub(r"\s+", "", label).lower():
                r = requests.get(href, allow_redirects=True)
                open(f"{PATH_WP}{year}/{value}.pdf", 'wb').write(r.content)
                break  # Stop checking once matched





In [ ]:
s="Horizon Europe Work programme (2025) - Infrastructures"

for key, value in files_to_load.items():
    if key in re.sub(r"\s+", "", s).lower():
        print(value)
#             result_dict[value] = href
#             break  # Stop checking once matched

In [ ]:
wp_year='2025'
url=f'https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe/horizon-europe-work-programmes_en#pre-publication-of-work-programme-{wp_year}'
# url='https://research-and-innovation.ec.europa.eu/funding/funding-opportunities/funding-programmes-and-open-calls/horizon-europe/horizon-europe-work-programmes_en#pre-publication-of-work-programme-2025'
from constant_vars import FRAMEWORK

files_to_load = {"infrastructures":"infra", 
                "cluster1":"health",
                "cluster2":"cluster2",
                "cluster3":"cluster3",
                "cluster4":"cluster4",
                "cluster5":"cluster5",
                "cluster6":"cluster6",
                "uropeaninnovationecosystems":"eie",
                "widera":"widera",
                "missions":"mission",
                "neb":"neb"
                }

# if load_wp==True:
wp_load(url, wp_year, files_to_load)

In [ ]:
calls_by_wp=[]
for k,v in files_to_load.items():
    doc = fitz.open(f"{PATH_WP}{wp_year}/{v}.pdf")
    all_text = chr(12).join([page.get_text() for page in doc])
    all_text = all_text.replace('\n','')
    all_text = all_text.strip()
    # print(all_text)
    match = re.search(r"table of contents(.*?)budget", all_text, re.DOTALL | re.IGNORECASE)
    if match:
        result = match.group(1).strip()
        search_text = r'HORIZON-[^:\s\\\/\)]*'
        l=re.findall(search_text, result)
        l=list(set(l))
        res={'year':wp_year,
        'wp':v,
        'calls':list(set(l))}
        calls_by_wp.append(res)
pd.to_pickle(pd.DataFrame(calls_by_wp), open(f"{PATH_SOURCE}{FRAMEWORK}/calls_by_wp.pkl", 'wb'))

In [ ]:
from config_path import PATH_SOURCE, PATH_WP
from constant_vars import FRAMEWORK
import time, pandas as pd, requests, fitz, re
pdf=[]
doc = fitz.open(f"{PATH_SOURCE}{FRAMEWORK}/MSCA_Keywords.pdf")
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()  # default: plain text extraction
    # print(f"--- Page {page_num + 1} ---")
    # print(text)


In [ ]:
import pdfplumber
import pandas as pd
import re

all_rows = []

with pdfplumber.open(f"{PATH_SOURCE}{FRAMEWORK}/MSCA_Keywords.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_tables()
        print(text)
        # if not text:
        #     continue
        # lines = text.split('\n')
        # print(lines)
#         for line in lines:
#             # Filtrer les lignes de bas de page (ex: "Page 1", "1", etc.)
#             if re.match(r"^\s*Page\s*\d+\s*$", line, re.IGNORECASE) or re.match(r"^\s*\d+\s*$", line):
#                 continue

#             # Découper par espace logique ou adapter ici
#             parts = line.strip().split()
#             all_rows.append(parts)

# # Normaliser toutes les lignes pour avoir le même nombre de colonnes
# max_cols = max(len(row) for row in all_rows)
# normalized = [row + [""] * (max_cols - len(row)) for row in all_rows]

# # Créer un DataFrame
# df = pd.DataFrame(normalized)

In [ ]:
test=f"{PATH_SOURCE}{FRAMEWORK}/msca_Keywords.txt"

In [ ]:
# def parse_and_fill_text_to_dataframe():
import numpy as np, pandas as pd
from config_path import PATH_SOURCE
from constant_vars import FRAMEWORK
test=f"{PATH_SOURCE}{FRAMEWORK}/msca_Keywords.txt"
with open(test, "r", encoding="utf-8") as f:
    raw_text = f.read()

lines = [line.strip() for line in raw_text.strip().split('\n') if line.startswith('|')]
rows = [line.strip('|').split('|') for line in lines]

clean_filtered_rows = [
    [cell.strip() for cell in row]
    for row in rows
    if not any('---' in cell or 'scientific panel' in cell.lower() for cell in row)
]

# Step 3: Use the original header (before filtering and cleaning)
header = [cell.strip() for cell in rows[0]]

# Step 4: Create DataFrame
df = pd.DataFrame(clean_filtered_rows, columns=header)


# # Step 4: Forward-fill missing values
df = df.replace('', np.nan).fillna(method='ffill').loc[~df["Level 2 keywords"].isnull()]

# Extract the part before the parenthesis
df["panel_regroupement_name"] = df["Scientific panel"].str.extract(r"^(.*?)\s*\(")

# Optional: also extract the code as before
df["panel_regroupement_code"] = df["Scientific panel"].str.extract(r"\((.*?)\)")

df[["panel_code_1", "panel_name_1"]] = df["Level 1 keywords"].str.split('-', n=1, expand=True)

df = (df[['panel_regroupement_code', 'panel_regroupement_name', 'panel_code_1', 'panel_name_1', 'Level 2 keywords']]
      .rename(columns={"Level 2 keywords":"panel_keywords"})
)
# print(df)
json_output_path = "data_files/msca_keywords.json"
df.to_json(json_output_path, orient="records", indent=2, force_ascii=False)

In [ ]:
df

In [ ]:
from functions_shared import work_csv
work_csv(df, 'msca_panel')

In [ ]:
df